In [1]:
%matplotlib inline
from collections import defaultdict
from constants import *
from os import listdir, makedirs
from os.path import join, isfile, exists, dirname
import gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from tqdm._tqdm_notebook import tqdm_notebook
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import Word2Vec, Doc2Vec
from utils import load

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

tqdm_notebook.pandas()
pd.options.display.max_columns = 30

-----

#### Loading topics and labels

In [8]:
args = [100, 'e42', 'ref', 'minimal']
t = 'topics'
l = 'labels'
dewat = load('dewa1', t, *args)
dewal = load('dewa1', t, *args)
et = load('e', t, *args)
el = load('e', l, *args)
fat = load('fa', t, *args)
fal = load('fa', l, *args)
fot = load('fo', t, *args)
fol = load('fo', l, *args)
ot = load('o', t, *args)
ol = load('o', l, *args)
pt = load('p', t, *args)
pl = load('p', l, *args)


Reading ../data/preprocessed/LDAmodel/noun/Reranker/dewac_nbfiles01_topic-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/dewac_nbfiles01_topic-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/Europarl_topic-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/Europarl_label-candidates_full.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/FAZ_topic-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/FAZ_label-candidates_full.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/FOCUS_topic-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/FOCUS_label-candidates_full.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/OnlineParticipation_topic-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/OnlineParticipation_label-candidates_full.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/PoliticalSpeeches_topic-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/Rerank

In [13]:
dewat = pd.read_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'dewac_nbfiles01_default_topics.csv'), index_col=0)
dewal = pd.read_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'dewac_nbfiles01_default_labels.csv'), index_col=0)
et = pd.read_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'Europarl_default_topics.csv'), index_col=0)
el = pd.read_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'Europarl_default_labels.csv'), index_col=0)
fat = pd.read_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'FAZ_default_topics.csv'), index_col=0)
fal = pd.read_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'FAZ_default_labels.csv'), index_col=0)
fot = pd.read_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'FOCUS_default_topics.csv'), index_col=0)
fol = pd.read_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'FOCUS_default_labels.csv'), index_col=0)
ot = pd.read_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'OnlineParticipation_default_topics.csv'), index_col=0)
ol = pd.read_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'OnlineParticipation_default_labels.csv'), index_col=0)
pt = pd.read_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'PoliticalSpeeches_default_topics.csv'), index_col=0)
pl = pd.read_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'PoliticalSpeeches_default_labels.csv'), index_col=0)

dewat

,term0,term1,term2,term3,term4,term5,term6,term7,term8,term9,cv_score,count_wiki_titles,cv_rank,wiki_rank,rank
24,Tier,Hund,Jäger,Jagd,Katze,Fuchs,Vogel,Mensch,Wald,Haustier,0.825688,10,93.0,78.5,171.5
12,Musik,Band,Bühne,Publikum,Lied,Konzert,Musiker,Theater,Künstler,Stück,0.809177,10,91.0,78.5,169.5
68,Artikel,Stern,Erde,Planet,KM,Link,Komet,Quelle,NASA,Sonne,0.795549,10,88.0,78.5,166.5
90,Gott,Jesus,Herr,Mensch,Glaube,Leben,Christus,Christ,Volk,Vater,0.775762,10,85.0,78.5,163.5
45,Uhr,Quelle,Dienstag,Freitag,Schröder,Montag,Spiegel,Mittwoch,Donnerstag,Samstag,0.757976,10,83.0,78.5,161.5
6,Kirche,Beamter,Bischof,Papst,Zeit,Kardinal,Katholische Kirche,Rom,Priester,Erzbischof,0.740925,10,78.0,78.5,156.5
32,Krieg,USA,Volk,Staat,Irak,Land,Welt,Regierung,Friede,Europa,0.726734,10,75.0,78.5,153.5
30,Deutschland,Juden,Deutsch,Hitler,Opfer,Berlin,Nazi,Pol,Krieg,Nationalsozialismus,0.716404,10,71.0,78.5,149.5
58,Akademie,Potsdam,Sachs,Magdeburg,Akademie der Wissenschaften,Kirche,Sachsen-Anhalt,Thüringen,Erfurt,Brandenburg,0.703753,10,69.0,78.5,147.5
83,Prozent,Euro,Bank,Unternehmen,Million,Geld,Anleger,Deutschland,Fond,Aktie,0.690818,10,67.0,78.5,145.5


-----

### unlemmatize topic terms and labels and rank topics

In [2]:
# map lemmatization back to unnormalized text
phrases = load('phrases', 'minimal')
phrases

Reading ../data/preprocessed/dewiki_phrases_lemmatized.pickle


token
Alan_Smithee                                                                                  Alan_Smithee
Actinium                                                                                          Actinium
ANG_Lee                                                                                            Ang_Lee
Anschluss                                                                                        Anschluss
Aussagenlogik                                                                                Aussagenlogik
Anthony_Minghella                                                                        Anthony_Minghella
us-amerikanisch_Film                                                                US-amerikanischer_Film
Vorsatz_für_Maßeinheit                                                           Vorsätze_für_Maßeinheiten
Abkürzungen/Gesetz_und_Recht                                                 Abkürzungen/Gesetze_und_Recht
Liste_von_Unternehmen_mit_Namen

In [ ]:
wikt = load('wikt', 'lemmap')
wikt = wikt.set_index('Lemma')
'Hallo' in wikt.index

In [28]:
wikt.loc['Schade']

,Form,POS
Lemma,,
Schade,Schaden,Noun
Schade,Schäden,Noun
Schade,Schadens,Noun
Schade,Schade,Noun


In [5]:
lmo = load('o', 'lemmap')
lmo

Reading ../data/preprocessed/OnlineParticipation_lemmatization_map.pickle


token                                                                                                   
A                                                    A                                                       86
                                                     a                                                       81
A-Jugend                                             A-Jugend                                                 1
A-Klasse                                             A-Klasse                                                 1
A-Orchester                                          A-Orchester                                              2
A-Platz                                              A-Platz                                                  1
A-Promi                                              A-Promis                                                 1
A-Soziale                                            a-soziales                                               1

In [38]:
lmo['Gully'].index[0]
'Gully' in lmo

True

In [71]:
def unlemmatize(token, lemmap=None):
    #print()
    if lemmap is not None and token in lemmap:
        #print('token in lemmap')
        #print(lemmap[token])
        word = lemmap[token].index[0]
        #print('   ', token, '->', word)
    elif token in phrases.index:
        # print('token in phrases')
        words = phrases[token]
        if isinstance(words, str):
            word = words
        else:
            if token in words.values:
                word = token
            else:
                wc = words.value_counts()
                word = wc.index[0]
    # elif token.lower() in wikt:
    #     print('token in wikt')
    #     print(wikt[token])
    #     word = 'todo'
    else:
        print('not in index')
        word = token
        print('   ', token, '->', word)
    word = word.replace('_', ' ')
    return word

In [72]:
collected = []
for dataset in ['O', 'P', 'E', 'FA', 'FO', 'dewa1']:
    lemmap = load(dataset, 'lemmap')
    for metric in ['ref', 'c_npmi']:
        params = [dataset, metric, 100, 'e42', 'minimal']
        l = load('labels', *params).applymap(unlemmatize)
        t = load('topics', *params)
        s = load('scores', *params)
        t['cv_score'] = s.c_v_eval
        # wiktionary would be better than wikipedia
        t['count_regular_terms'] = t.loc[:, 'term0':'term9'].applymap(lambda x: x in phrases.index or x in wikt.index).sum(axis=1)
        t['cv_rank'] = t.cv_score.rank()
        t['wiki_rank'] = t.count_regular_terms.rank()
        t['rank'] = t.cv_rank + t.wiki_rank
        t.loc[:, 'term0':'term9'] = t.loc[:, 'term0':'term9'].applymap(lambda x: unlemmatize(x, lemmap))
        #t = t.sort_values('rank', ascending=False)
        #l = l.iloc[t.index, :]
        collected.append((dataset, metric, t, l))

Reading ../data/preprocessed/OnlineParticipation_lemmatization_map.pickle
Reading ../data/preprocessed/LDAmodel/noun/Reranker/OnlineParticipation_label-candidates_full.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/OnlineParticipation_topic-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/OnlineParticipation_evaluation-scores.csv
not in index
    2_. -> 2_.
not in index
    Stadt_Bonn -> Stadt_Bonn
not in index
    1_. -> 1_.
not in index
    Laura_Welsch -> Laura_Welsch
not in index
    Vogelsanger_Straße -> Vogelsanger_Straße
not in index
    Friedrich_Breuer_Straße -> Friedrich_Breuer_Straße
not in index
    3_. -> 3_.
not in index
    Stadt_Bonn -> Stadt_Bonn
not in index
    Kölner_Straße -> Kölner_Straße
not in index
    @#4.1 -> @#4.1
Reading ../data/preprocessed/LDAmodel/noun/Reranker/OnlineParticipation_label-candidates_full.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/OnlineParticipation_topic-candidates.csv
Reading ../data/preprocessed/L

In [90]:
dic = defaultdict(dict)

for c in collected:
    print(c[0], c[1])
    t = c[2].sort_index()
    l = c[3].sort_index()
    tl = t.join(l)
    tl = tl[tl.count_regular_terms >= 9]
    tl = tl.sort_values('cv_rank', ascending=False)
    tl = tl[tl.cv_score > 0.4]
    dic[c[0]][c[1]] = tl
    print(len(tl))


O ref
39
O c_npmi
56
P ref
69
P c_npmi
90
E ref
52
E c_npmi
74
FA ref
81
FA c_npmi
80
FO ref
78
FO c_npmi
84
dewa1 ref
85
dewa1 c_npmi
90


In [91]:
dic['news']['ref'] = pd.concat([dic['FA']['ref'], dic['FO']['ref']])
dic['pol']['ref'] = pd.concat([dic['O']['ref'], dic['P']['ref']])

In [115]:
dfo = dic['O']['ref'].reset_index(drop=True).assign(dataset=DATASETS['O']).set_index('dataset', append=True)
dfo

,,term0,term1,term2,term3,term4,term5,term6,term7,term8,term9,cv_score,count_regular_terms,cv_rank,wiki_rank,rank,...,label5,label6,label7,label8,label9,label10,label11,label12,label13,label14,label15,label16,label17,label18,label19
,dataset,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,OnlineParticipation,Beitrag,Dank,Kategorie,Verständnis,Vorschlag,Thema,Auswertung,Themenbereich,Moderation,Gast,0.974564,10,100.0,76.0,176.0,...,RSH-Gold,Satellite Awards 2017,Fernsehsendung,Ethik in der Medizin,Sigmund-Freud-Preis für wissenschaftliche Prosa,Wissenschaftsbuch des Jahres,Fachgebiet,Popularisierung,Wissenschaftliche Arbeit,Interaktiv,Veranstaltung,Workshop,Deutscher Zukunftspreis,Österreichischer Radiopreis,Aspekt
1,OnlineParticipation,Moderation,Grüße,Kommentar,Dank,Hallo,Hinweis,Dialog,Diskussion,Aussagen,Teilnehmer,0.875000,10,99.0,76.0,175.0,...,Show,Vortrag,Bericht,Schilderung,Rhetorische Frage,Erörterung,Offener Brief,Debatte,Zitat,Äußerung,Podiumsdiskussion,Ratschlag,SonntagAbend,Trockener Humor,Trackback
2,OnlineParticipation,Kinder,Schulen,Jugendliche,Schüler,Eltern,Kitas,Kindergärten,Grundschule,Familien,Angebot,0.858925,10,98.0,76.0,174.0,...,Förderschule,Regelschule,Ganztagsschule,Hauptschule,Kindertagesstätte,Kindergarten,Privatschule,Grundschulförderklasse,Offene Ganztagsschule,Sonderschule,Oberschule,Gesamtschule,Sekundarschule,Schuljahr,Realschule
3,OnlineParticipation,Radfahrer,Fußgänger,Ampel,Autofahrer,Ampelschaltung,Richtung,Grün,Fußgängerampel,Radler,Spur,0.849684,9,97.0,36.0,133.0,...,Beschleunigungsstreifen,Fußgängerfurt,Fahrradstraße,Kreisverkehr,Straßenverkehr,Bahnübergang,Abbiegen,Fußgängerüberweg,Straßenkreuzung,Grünpfeil,Zweispurig,Einbahnstraße,Unterführung,Verkehrsinsel,Nothaltebucht
4,OnlineParticipation,Radfahrer,Straße,Autofahrer,Fahrradstraße,Richtung,Fahrradfahrer,Fahrradweg,Autos,Fußgänger,Radweg,0.827118,10,96.0,76.0,172.0,...,Kraftfahrstraße,Radfahrer,Einbahnstraße,Straßenverkehr,Unterführung,Parkstand,RAVeL-Netz,Beschleunigungsstreifen,Kreisverkehr,Fußgängerbrücke,Gehweg,Bahnkörper,Gegenverkehr,Wohnstraße,Milseburgradweg
5,OnlineParticipation,Richtung,Radweg,Einmündung,Bürgersteig,Radfahrer,Einfahrt,Bordstein,Süden,Fahrtrichtung,Norden,0.763108,9,95.0,36.0,131.0,...,Straßenbrücke,Fußgängerbrücke,Effnerstraße,Kreisverkehr,Bundesstraße 277,Staatsstraße 2308,Zweispurig,Fahrweg,Route 31,Streckengleis,Anschlussstelle,Unterführung,Wendeschleife,State Highway 29,Route 23
6,OnlineParticipation,Busse,Bahn,Linie,ÖPNV,Strecke,Verbindung,Haltestelle,Stau,Buslinie,Busbahnhof,0.738060,10,94.0,76.0,170.0,...,Nahverkehr,Schnellbus,Schienenersatzverkehr,Umsteigen,Endbahnhof,Verknüpfungspunkt,Haltestelle,Straßenbahn,Wendeschleife,Nachtverkehr,RegioBus Hannover,Stichstrecke,S-Bahn,Regionalzug,Stadtbahn
7,OnlineParticipation,Bonn,Oper,Kultur,Stadt,Köln,Geld,Stadt Bonn,Theater,Bürger,Beethoven,0.731620,9,93.0,36.0,129.0,...,Deutsche Bühne,Bochumer Symphoniker,Bayerische Staatsoper,Kammeroper Köln,Philharmonie,Deutsche Oper am Rhein,Kleines Theater,Stadttheater,Sinfonieorchester Münster,Neue Stücke aus Europa,Aufführung,Konzerthaus,Beethoven Orchester Bonn,Bühnenwerk,Beethovenhalle
8,OnlineParticipation,Radweg,Radfahrer,Fahrbahn,Fußgänger,Weg,Straße,Stelle,Bushaltestelle,Zustand,Gefahr,0.691370,10,91.0,76.0,167.0,...,Haltestellenkap,Gegenverkehr,Verkehrsinsel,Busschleuse,Staatsstraße 2308,Parkstand,Zweispurig,Fahrweg,Gehweg,Verkehrsteilnehmer,Bahnübergang,Fußgängerfurt,Einbahnstraße,Wirtschaftsweg,Milseburgradweg


In [116]:
dfd = dic['dewa1']['ref'].reset_index(drop=True).assign(dataset='dewac').set_index('dataset', append=True).query('cv_score > 0.5')
dfd

,,term0,term1,term2,term3,term4,term5,term6,term7,term8,term9,cv_score,count_regular_terms,cv_rank,wiki_rank,rank,...,label5,label6,label7,label8,label9,label10,label11,label12,label13,label14,label15,label16,label17,label18,label19
,dataset,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,dewac,Hilfe,Hausaufgaben,Lehrer,Geometrie,KeepSchool,Klassenarbeit,Abitur,Algebra,Rechnen,Biologie,0.887623,9,99.0,16.5,115.5,...,Analytische Geometrie,Fakultativer Unterricht,Abstrakte Algebra,Berufsgrundschuljahr,Realgymnasium,Kunsterziehung,Vorkurs,Algebraische Funktion,Pädagogische Psychologie,Wirtschaftsmathematik,Sekundarstufe I,Arithmetik,Lernbehindertenpädagogik,Fachbereich,Musikdidaktik
1,dewac,Zivildienst,Wehrpflicht,Bundeswehr,Wehrpflichtige,Monate,Dienst,Kriegsdienstverweigerer,Soldaten,Zivildienstleistenden,Einberufung,0.876197,9,98.0,16.5,114.5,...,Soldat auf Zeit,Freiwilliger Wehrdienst,Soldatengesetz,Zivildienst,Wehrsold,Wehrersatzdienst,Zivildienstgesetz,Rekrut,Soldatenversorgungsgesetz,Reserveoffizier,Gebirgsjäger,Dreijährig-Freiwilliger,Totalverweigerung,Wehrpflicht in Deutschland,Wehrsoldgesetz
2,dewac,Österreich,Wien,Österreicher,Salzburg,Jänner,Linz,FPÖ,ÖVP,Graz,KPÖ,0.863826,9,97.0,16.5,113.5,...,Niederösterreich,Steiermark,Oberösterreich,Wien,Salzburg,Niederösterreichische Landesregierung,Freiheitliche Partei Österreichs,Linz,Oberösterreichischer Landtag,Villach,Kärnten,Kapfenberg,Wiener Neustadt,Innsbruck,Vorarlberger Volkspartei
3,dewac,Lebenspartnerschaft,Ehe,Lebenspartner,Schwule,Partner,Lesben,Ehegatten,Abs.,Paare,Partnerschaft,0.841291,10,95.0,62.5,157.5,...,Lesbisch,Scheidung,Eherecht,Gütertrennung,Homosexualität,Ehescheidung,Regenbogenfamilie,Lebensformenpolitik,Abstandsgebot,Sexuelle Identität,Partnerschaftsgesetz,Eingetragene Partnerschaft,Sexualpartner,Ehepakt,Elterliche Sorge
4,dewac,Israel,Palästinenser,Palästina,Juden,Soldaten,Israelis,Haus,Jerusalem,Gebiet,Arafat,0.838946,9,94.0,16.5,110.5,...,Palästinakrieg,Israelische Siedlung,Arabische Israelis,Westjerusalem,Nahostkonflikt,Zionismus,Südlicher Libanon,Sechstagekrieg,Ostjerusalem,Israelisch besetzte Gebiete,Hamas,Zweite Intifada,Israeli,Arabische Befreiungsarmee,Kirjat Arba
5,dewac,Tiere,Hund,Jäger,Jagd,Katze,Fuchs,Vogel,Menschen,Wald,Haustiere,0.825688,10,93.0,62.5,155.5,...,Nutztier,Kleine Indische Zibetkatze,Kulturflüchter,Kleinsäuger,Haustier,Dingo,Kusus,Riesenbeutelmarder,Haushund,Beutegreifer,Lungenwurm,Kaninchen,Fleischfresser,Indische Gazelle,Pampashirsch
6,dewac,Abs.,Arbeitnehmer,Nr.,Arbeitgeber,Art,Kläger,Urteil,GG,Regelung,BAG,0.819246,10,92.0,62.5,154.5,...,Vorrang des Gesetzes,Teilurteil,Bundesrechtsanwaltsordnung,Siebtes Buch Sozialgesetzbuch,Beschlussverfahren,Gestaltungsklage,Prozessgericht,Sozialgerichtsgesetz,Abwicklungsvertrag,Landesbeamtengesetz,Tarifvertragspartei,Bundespersonalvertretungsgesetz,Günstigkeitsprinzip,Personalvertretungsgesetz,Zulässigkeit
7,dewac,Musik,Band,Bühne,Publikum,Lied,Konzert,Musiker,Theater,Künstler,Stück,0.809177,10,91.0,62.5,153.5,...,Klassische Musik,Rockband,Auftritt,Bühnenwerk,Show,Singer-Songwriter,Bühnenmusik,Darsteller,Musik sein,Arrangeur,Liedermacher,Gastspiel,Soundtrack,Gitarrist,Musiker
8,dewac,SPD,Grünen,CDU,Partei,Berlin,Bundestag,Fraktion,Bundesregierung,FDP,Abgeordneten,0.804171,10,90.0,62.5,152.5,...,Landtagsabgeordneter,Freie Wähler,Landtagswahlen in Nordrhein-Westfalen,Landtagswahl im Saarland 2009,Rot-grüne Koalition,CDU/CSU-Bundestagsfraktion,FDP Brandenburg,Parlamentarischer Geschäftsführer,Landesvorstand,Sozialdemokratische Partei Deutschlands,Fraktionsgemeinschaft,Hessischer Landtag,SPD-Parteivorstand,Bundestagswahl 1983,Abgeordneter


In [117]:
dffa = dic['FA']['ref'].sort_values('cv_score', ascending=False).reset_index(drop=True).assign(dataset='news').set_index('dataset', append=True).query('cv_score > 0.5')
dffa

,,term0,term1,term2,term3,term4,term5,term6,term7,term8,term9,cv_score,count_regular_terms,cv_rank,wiki_rank,rank,...,label5,label6,label7,label8,label9,label10,label11,label12,label13,label14,label15,label16,label17,label18,label19
,dataset,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,news,SPD,Union,CDU,Partei,Schulz,Große Koalition,Martin Schulz,Merkel,Sozialdemokraten,Bundestagswahl,0.922986,9,100.0,20.0,120.0,...,Landtagswahlen in Nordrhein-Westfalen,Bundestagswahl 1969,Rot-grüne Koalition,Die Linke,Große Koalition,Bundestagswahl 1983,Kanzlerkandidat,Kabinett Kiesinger,Landtagswahlen in Thüringen,Bundestagswahl 1976,Bundestagswahl 2017,Wahlbündnis,Freie Wähler,Katrin Budde,Aktion Demokratischer Fortschritt
1,news,Spiel,Spieler,Kovac,Trainer,Eintracht,Mannschaft,Saison,Minuten,Fußball,Mainz,0.909523,9,99.0,20.0,119.0,...,Feldspieler,FSV Frankfurt,VfL Wolfsburg,FC Bayern München,Innenverteidiger,Stuttgarter Kickers,Karlsruher SC,Fortuna Düsseldorf,Hannover 96,VfL Osnabrück,Nationaltrainer,Eintracht Braunschweig,Spielertrainer,Tabellenführer,VfL Bochum
2,news,Auto,Fahrzeuge,Volkswagen,VW,Daimler,Hersteller,BMW,Autohersteller,Fahrverbote,Diesel,0.906389,10,98.0,65.5,163.5,...,Audi,General Motors,Hyundai,Elektroauto,VW Touareg,Volvo,Renault,BMW 3er,Opel,Fiat,Porsche,Chevrolet,Kleinstwagen,Allradantrieb,Rennwagen
3,news,Russland,Putin,Moskau,Ukraine,Sanktionen,Präsident,Wladimir,Geheimdienste,Russen,Kreml,0.894025,9,96.0,20.0,116.0,...,Sowjetunion,Sicherheitsrat der Russischen Föderation,Andrei,Ukrain,Aserbaidschanische Sozialistische Sowjetrepublik,Krimkrise,Leonid Kutschma,Kiew,Krim,Russisches Kaiserreich,Sowjetrussland,Nischni Nowgorod,Bolschewiki,Aserbaidschan,Abchasien
4,news,Republikaner,Demokraten,Senat,Senator,Kongress,Bundesstaat,Gesetz,Präsident,Repräsentantenhaus,Obamacare,0.889050,10,95.0,65.5,160.5,...,Senat von Mississippi,Senat von Colorado,Repräsentantenhaus der Vereinigten Staaten,Senat von Maine,Senat von Utah,Senat von Connecticut,Senat von Louisiana,Senat von Alabama,Republikanische Partei,Senat von Kentucky,Senat von Massachusetts,Senat von Michigan,Senat von New Mexico,Repräsentantenhaus von Hawaii,Repräsentantenhaus von Utah
5,news,Israel,Palästinenser,Juden,Jerusalem,Netanjahu,Botschaft,Hauptstadt,Ägypten,Gazastreifen,Israelis,0.888318,9,94.0,20.0,114.0,...,Hamas,Nahostkonflikt,Arabische Israelis,Israelische Siedlung,Zweite Intifada,Israeli,Palästinensische Autonomiegebiete,Westjerusalem,Sechstagekrieg,Palästinakrieg,Benjamin Netanjahu,Ostjerusalem,Südlicher Libanon,Israelisch besetzte Gebiete,Gaza
6,news,London,Großbritannien,Brexit,Briten,May,Premierministerin,EU,Theresa May,Regierung,Austritt,0.888202,10,93.0,65.5,158.5,...,Conservative Party,Chris Patten,Kabinett Cameron I,Theresa May,Britische Unterhauswahl 1997,Königreich Großbritannien,Regierungschef,Liberal Democrats,Außengrenzen der Europäischen Union,Liam Fox,Europäische Wirtschaftsgemeinschaft,Centre for Policy Studies,Britisches Weltreich,Winston Churchill,Premierminister
7,news,Spanien,Barcelona,Madrid,Unabhängigkeit,Katalonien,Puigdemont,Referendum,Regierung,Belgien,Region,0.871090,9,91.0,20.0,111.0,...,Junts pel Sí,Francisco Franco,Aragonien,Montmeló,Carles Puigdemont,Autonome Gemeinschaft Baskenland,Convergència i Unió,Esquerra Republicana de Catalunya,Zweite Spanische Republik,Sestao,Valencianische Gemeinschaft,Saragossa,Alt Urgell,Katalanen,Montcada i Reixac
8,news,Musik,Bühne,Band,Lied,Stück,Album,Publikum,Konzert,Musiker,Song,0.868389,10,90.0,65.5,155.5,...,Lied,Tracks,Kill Rock Stars,Singer-Songwriter,B-Seite,Tournee,Doppelalbum,Vocals,Buried Alive,Auftritt,Konzert,Klassische Musik,Aufführung,Coverversion,Gitarrist


In [118]:
dfp = dic['P']['ref'].sort_values('cv_score', ascending=False).reset_index(drop=True).assign(dataset='speeches').set_index('dataset', append=True).query('cv_score > 0.5')
dfp

,,term0,term1,term2,term3,term4,term5,term6,term7,term8,term9,cv_score,count_regular_terms,cv_rank,wiki_rank,rank,...,label5,label6,label7,label8,label9,label10,label11,label12,label13,label14,label15,label16,label17,label18,label19
,dataset,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,speeches,Opfer,Erinnerung,Auschwitz,Juden,Holocaust,Nationalsozialismus,Verbrechen,Gedenken,Denkmal,Sinti und Roma,0.901204,10,100.0,58.5,158.5,...,Vernichtungslager,Deportation,Völkermord,Massenmord,Antisemitismus,Yad Vashem,Verbrechen gegen die Menschlichkeit,Shoah,Kommunistisches Verbrechen,Verbrechen der Wehrmacht,Massengrab,Kleine Festung Theresienstadt,Endphaseverbrechen,Jüdisches Historisches Institut,Jom haScho’a
1,speeches,Film,Kino,Deutscher Film,Berlinale,Filmförderung,Produzenten,Erfolg,Euro,Regisseur,Millionen,0.771645,10,99.0,58.5,157.5,...,Satellite Awards 2017,Copenhagen International Film Festival,Hessischer Filmpreis,Akiva Goldsman,Peter Chelsom,Darsteller,Markus Goller,Preis der deutschen Filmkritik,Heinz Willeg,Deutscher Film,Wiedemann & Berg Filmproduktion,Independent-Film,Filmstarts,Filmproduktion,Filmpreis
2,speeches,Kultur,Naumann,Sender,Fernsehen,ARD,ZDF,Kommunikation,Programm,Öffentlich-rechtlicher Rundfunk,3sat,0.765007,9,98.0,10.0,108.0,...,NDR Fernsehen,ARD-alpha,Hörfunksendung,ZDF,Echo des Tages,Sender Freies Berlin,Bayerischer Rundfunk,Schweizer Fernsehen,ZDFinfo,ProSieben,Europamagazin,Drittes Fernsehprogramm,Tagesschau24,ARD Digital,Pay-TV
3,speeches,Ostdeutschland,DDR,Neue Länder,Deutsche Einheit,Einheit,Osten,Bundesländern,Westen,Ostdeutschen,Freiheit,0.763741,10,97.0,58.5,155.5,...,Deutsche Demokratische Republik,Wehrbereichskommando VII,Ländereinführungsgesetz,Bundesdeutsch,Beitrittsgebiet,Kreis Bad Freienwalde,Westzone,Ostdeutschland,Kreis Wittstock,Verwaltungsgemeinschaft Nordkreis Weimar,Land Brandenburg,Kreis Eberswalde,Landkreis Hagenow,Neue Länder,Deutscher Handballverband
4,speeches,Geschichte,Erinnerung,Opfer,Vergangenheit,Verantwortung,Gedenken,Deutschen,Gedenkstätte,Verbrechen,Überlebenden,0.750109,10,96.0,58.5,154.5,...,Weiße Rose Stiftung,Ungesühnt,KZ-Friedhof,Historisches Ereignis,Erinnerung,Massenmord,Grausamkeit,Gedenktag,Instytut Pamięci Narodowej,Kollektivschuld,Tote des Zweiten Weltkrieges,Erinnerungsort,Verbrechen gegen die Menschlichkeit,Kriegsopfer,Konzentrationslager
5,speeches,Ausstellung,Kunst,Museum,Künstler,Werk,Berlin,Bild,Kultur,Sammlung,Geschichte,0.712076,10,95.0,58.5,153.5,...,Kunstwerk,Werkschau,Privatsammlung,Kunsthistoriker,Kunstgewerbemuseum,Kunstausstellung,Wanderausstellung,Wechselausstellung,Ausstellung,Kunstmuseum,Kunstform,Angewandte Kunst,Neues Museum Nürnberg,Retrospektive,Saint Louis Art Museum
6,speeches,Literatur,Schriftsteller,Buch,Autoren,Kultur,Sprache,Werk,Geschichte,Berlin,Thomas Mann,0.698839,10,94.0,58.5,152.5,...,Dramatiker,Prosa,Schriftsteller,Gegenwartsliteratur,Erzählung,Romancier,Anthologie,Literaturwissenschaft,Kurzgeschichte,Literaturgeschichte,Roman,Belletristik,Essay,Sekundärliteratur,Kinderbuch
7,speeches,Universität,Hochschulen,Studenten,Studierenden,Bildung,Studium,Freiheit,Wissen,Professor,Forschung,0.695012,10,93.0,58.5,151.5,...,Dozent,Hochschule,Forschung und Lehre,Studium generale,Prorektor,Fachbereich,Pädagogische Hochschule,Master of Divinity,Medizinische Universität,Graduate School,Sozialwissenschaften,Studium,Humanwissenschaft,Kunsthochschule,Universität Maribor
8,speeches,Juden,Antisemitismus,Berlin,Ort,Geschichte,Synagoge,Paul Spiegel,Erinnerung,Verantwortung,Opfer,0.670613,10,92.0,58.5,150.5,...,Novemberpogrome 1938,Pogrom,Israelitische Kultusgemeinde,Neue Synagoge,Memorbuch,Ehemalige Synagogen,Jedwabne,Jüdischer Friedhof,Mahnmal,Synagoge,Antijudaismus,Gedenkstätte,Leo-Baeck-Preis,Große Synagoge,Judenpogrom


In [120]:
df = pd.concat([dfd, dffa, dfp, dfo])
df.loc[:, 'term0':'term9'].to_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'topics.csv'))
df.loc[:, 'label0':'label19'].to_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'labels.csv'))
df

,,term0,term1,term2,term3,term4,term5,term6,term7,term8,term9,cv_score,count_regular_terms,cv_rank,wiki_rank,rank,...,label5,label6,label7,label8,label9,label10,label11,label12,label13,label14,label15,label16,label17,label18,label19
,dataset,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,dewac,Hilfe,Hausaufgaben,Lehrer,Geometrie,KeepSchool,Klassenarbeit,Abitur,Algebra,Rechnen,Biologie,0.887623,9,99.0,16.5,115.5,...,Analytische Geometrie,Fakultativer Unterricht,Abstrakte Algebra,Berufsgrundschuljahr,Realgymnasium,Kunsterziehung,Vorkurs,Algebraische Funktion,Pädagogische Psychologie,Wirtschaftsmathematik,Sekundarstufe I,Arithmetik,Lernbehindertenpädagogik,Fachbereich,Musikdidaktik
1,dewac,Zivildienst,Wehrpflicht,Bundeswehr,Wehrpflichtige,Monate,Dienst,Kriegsdienstverweigerer,Soldaten,Zivildienstleistenden,Einberufung,0.876197,9,98.0,16.5,114.5,...,Soldat auf Zeit,Freiwilliger Wehrdienst,Soldatengesetz,Zivildienst,Wehrsold,Wehrersatzdienst,Zivildienstgesetz,Rekrut,Soldatenversorgungsgesetz,Reserveoffizier,Gebirgsjäger,Dreijährig-Freiwilliger,Totalverweigerung,Wehrpflicht in Deutschland,Wehrsoldgesetz
2,dewac,Österreich,Wien,Österreicher,Salzburg,Jänner,Linz,FPÖ,ÖVP,Graz,KPÖ,0.863826,9,97.0,16.5,113.5,...,Niederösterreich,Steiermark,Oberösterreich,Wien,Salzburg,Niederösterreichische Landesregierung,Freiheitliche Partei Österreichs,Linz,Oberösterreichischer Landtag,Villach,Kärnten,Kapfenberg,Wiener Neustadt,Innsbruck,Vorarlberger Volkspartei
3,dewac,Lebenspartnerschaft,Ehe,Lebenspartner,Schwule,Partner,Lesben,Ehegatten,Abs.,Paare,Partnerschaft,0.841291,10,95.0,62.5,157.5,...,Lesbisch,Scheidung,Eherecht,Gütertrennung,Homosexualität,Ehescheidung,Regenbogenfamilie,Lebensformenpolitik,Abstandsgebot,Sexuelle Identität,Partnerschaftsgesetz,Eingetragene Partnerschaft,Sexualpartner,Ehepakt,Elterliche Sorge
4,dewac,Israel,Palästinenser,Palästina,Juden,Soldaten,Israelis,Haus,Jerusalem,Gebiet,Arafat,0.838946,9,94.0,16.5,110.5,...,Palästinakrieg,Israelische Siedlung,Arabische Israelis,Westjerusalem,Nahostkonflikt,Zionismus,Südlicher Libanon,Sechstagekrieg,Ostjerusalem,Israelisch besetzte Gebiete,Hamas,Zweite Intifada,Israeli,Arabische Befreiungsarmee,Kirjat Arba
5,dewac,Tiere,Hund,Jäger,Jagd,Katze,Fuchs,Vogel,Menschen,Wald,Haustiere,0.825688,10,93.0,62.5,155.5,...,Nutztier,Kleine Indische Zibetkatze,Kulturflüchter,Kleinsäuger,Haustier,Dingo,Kusus,Riesenbeutelmarder,Haushund,Beutegreifer,Lungenwurm,Kaninchen,Fleischfresser,Indische Gazelle,Pampashirsch
6,dewac,Abs.,Arbeitnehmer,Nr.,Arbeitgeber,Art,Kläger,Urteil,GG,Regelung,BAG,0.819246,10,92.0,62.5,154.5,...,Vorrang des Gesetzes,Teilurteil,Bundesrechtsanwaltsordnung,Siebtes Buch Sozialgesetzbuch,Beschlussverfahren,Gestaltungsklage,Prozessgericht,Sozialgerichtsgesetz,Abwicklungsvertrag,Landesbeamtengesetz,Tarifvertragspartei,Bundespersonalvertretungsgesetz,Günstigkeitsprinzip,Personalvertretungsgesetz,Zulässigkeit
7,dewac,Musik,Band,Bühne,Publikum,Lied,Konzert,Musiker,Theater,Künstler,Stück,0.809177,10,91.0,62.5,153.5,...,Klassische Musik,Rockband,Auftritt,Bühnenwerk,Show,Singer-Songwriter,Bühnenmusik,Darsteller,Musik sein,Arrangeur,Liedermacher,Gastspiel,Soundtrack,Gitarrist,Musiker
8,dewac,SPD,Grünen,CDU,Partei,Berlin,Bundestag,Fraktion,Bundesregierung,FDP,Abgeordneten,0.804171,10,90.0,62.5,152.5,...,Landtagsabgeordneter,Freie Wähler,Landtagswahlen in Nordrhein-Westfalen,Landtagswahl im Saarland 2009,Rot-grüne Koalition,CDU/CSU-Bundestagsfraktion,FDP Brandenburg,Parlamentarischer Geschäftsführer,Landesvorstand,Sozialdemokratische Partei Deutschlands,Fraktionsgemeinschaft,Hessischer Landtag,SPD-Parteivorstand,Bundestagswahl 1983,Abgeordneter


In [119]:
len(dfd) + len(dffa) + len(dfp) + len(dfo)

235

In [70]:
b = collected[0][2]
#b = b.sort_values('count_wiki_titles', ascending=False)
b = b[b.count_wiki_titles >= 9]
b = b.sort_values('cv_rank', ascending=False)
b = b[b.cv_score > 0.4]
print(len(b))
b

25


,term0,term1,term2,term3,term4,term5,term6,term7,term8,term9,cv_score,count_wiki_titles,cv_rank,wiki_rank,rank
38,Beitrag,Dank,Kategorie,Verständnis,Vorschlag,Thema,Auswertung,Themenbereich,Moderation,Gast,0.974564,10,100.0,76.0,176.0
10,Moderation,Grüße,Kommentar,Dank,Hallo,Hinweis,Dialog,Diskussion,Aussagen,Teilnehmer,0.875000,10,99.0,76.0,175.0
42,Kinder,Schulen,Jugendliche,Schüler,Eltern,Kitas,Kindergärten,Grundschule,Familien,Angebot,0.858925,10,98.0,76.0,174.0
94,Radfahrer,Straße,Autofahrer,Fahrradstraße,Richtung,Fahrradfahrer,Fahrradweg,Autos,Fußgänger,Radweg,0.827118,10,96.0,76.0,172.0
37,Busse,Bahn,Linie,ÖPNV,Strecke,Verbindung,Haltestelle,Stau,Buslinie,Busbahnhof,0.738060,10,94.0,76.0,170.0
19,Radweg,Radfahrer,Fahrbahn,Fußgänger,Weg,Straße,Stelle,Bushaltestelle,Zustand,Gefahr,0.691370,10,91.0,76.0,167.0
2,Vorschlag,Bürgerhaushalt,Redaktion,Bürger,Thema,Verwaltung,Bezirksvertretung,Grund,Stadt,Möglichkeit,0.679243,10,90.0,76.0,166.0
23,Euro,Jahren,Kosten,Millionen,Mio.,Köln,Stadt,Ausgaben,Vorteil,Problem,0.655641,10,88.0,76.0,164.0
25,Stadt,Einnahmen,Unterstützung,Köln,Umsetzung,Lebensqualität,Steuerzahler,Bürger,Angebot,Stadt Köln,0.631783,10,85.0,76.0,161.0
97,Gehweg,Park,Ordnungsamt,Polizei,Reihe,Verbot,Halteverbot,LKW,Stadt,Bereich,0.584009,10,82.0,76.0,158.0


In [54]:
a.sort_values('count_wiki_titles', ascending=False)

,term0,term1,term2,term3,term4,term5,term6,term7,term8,term9,cv_score,count_wiki_titles,cv_rank,wiki_rank,rank
2,Vorschlag,Bürgerhaushalt,Redaktion,Bürger,Thema,Verwaltung,Bezirksvertretung,Grund,Stadt,Möglichkeit,0.679243,10,90.0,96.0,186.0
72,Meinung,Zukunft,Ehrenfeld,Köln,Stadt,Jahren,Großstadt,Priorität,Ausstellungen,Sportstätten,0.451980,10,65.0,96.0,161.0
0,Bad Godesberg,Lage,Hauptbahnhof,Stadt,Pflastersteine,GAG,Rathaus,Berlin,Innenstadt,City,0.375969,10,41.0,96.0,137.0
91,Qualität,Förderung,Integration,Stadt,Treppe,Trasse,Aufzug,Lust,Jugendamt,Projekt,0.295968,10,12.0,96.0,108.0
4,Familien,Gruppen,Köln,Geld,Stadt Köln,Mann,Einrichtung,Hilfe,Blücherpark,Lärmschutzwand,0.375983,10,42.0,96.0,138.0
80,Mitarbeiter,Amt,Verkehrssicherheit,Erhöhung,Argumente,Stadt,Staat,Geld,Stelle,Aufgabe,0.440795,10,62.0,96.0,158.0
25,Stadt,Einnahmen,Unterstützung,Köln,Umsetzung,Lebensqualität,Steuerzahler,Bürger,Angebot,Stadt Köln,0.631783,10,85.0,96.0,181.0
81,Flächen,Frauen,Stadt,Initiative,Freibad,Bürger,Garten,Wort,Pflege,Begrünung,0.425540,10,59.0,96.0,155.0
14,Museum,Eindruck,Informationen,Gefühl,Gegenteil,Politik,Stadt,Regel,Jahren,Lösung,0.470960,10,70.0,96.0,166.0
7,Nutzung,Grünflächen,Veranstaltungen,Rheinaue,Stadt,LED,Bebauung,Veranstalter,Einfluss,Akademie,0.329623,9,20.0,77.0,97.0


In [22]:
for tpl in collected:
    variant = 'default' if tpl[1] == 'ref' else 'rerank'
    file = f'{DATASETS[tpl[0]]}_{variant}'
    tpl[2].to_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', file + '_topics.csv'))
    tpl[3].to_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', file + '_labels.csv'))

In [4]:
dataset, metric = 'dewac1', 'ref'
params = [dataset, metric, 100, 'e42', 'minimal']
l = load('labels', *params).applymap(unlemmatize)
t = load('topics', *params)
s = load('scores', *params)
t['cv_score'] = s.c_v_eval
# wiktionary would be better than wikipedia
t['count_wiki_titles'] = t.loc[:, 'term0':'term9'].applymap(lambda x: x in phrases.index).sum(axis=1)
t['cv_rank'] = t.cv_score.rank()
t['wiki_rank'] = t.count_wiki_titles.rank()
t['rank'] = t.cv_rank + t.wiki_rank
t.loc[:, 'term0':'term9'] = t.loc[:, 'term0':'term9'].applymap(unlemmatize)
t = t.sort_values('rank', ascending=False)
t

Reading ../data/preprocessed/LDAmodel/noun/Reranker/dewac_nbfiles01_label-candidates_full.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/dewac_nbfiles01_topic-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/dewac_nbfiles01_evaluation-scores.csv


,term0,term1,term2,term3,term4,term5,term6,term7,term8,term9,cv_score,count_wiki_titles,cv_rank,wiki_rank,rank
24,Tier,Hund,Jäger,Jagd,Katze,Fuchs,Vogel,Mensch,Wald,Haustier,0.825688,10,93.0,78.5,171.5
12,Musik,Band,Bühne,Publikum,Lied,Konzert,Musiker,Theater,Künstler,Stück,0.809177,10,91.0,78.5,169.5
68,Artikel,Stern,Erde,Planet,KM,Link,Komet,Quell,NASA,Sonne,0.795549,10,88.0,78.5,166.5
90,Gott,Jesus,Herr,Mensch,Glaube,Leben,Christus,Christ,Volk,Vater,0.775762,10,85.0,78.5,163.5
45,Uhr,Quell,Dienstag,Freitag,Schröder,Montag,Spiegel,Mittwoch,Donnerstag,Samstag,0.757976,10,83.0,78.5,161.5
6,Kirche,Beamter,Bischof,Papst,Zeit,Kardinal,katholisch Kirche,Rom,Priester,Erzbischof,0.740925,10,78.0,78.5,156.5
32,Krieg,USA,Volk,Staat,Irak,Land,Welt,Regierung,Friede,Europa,0.726734,10,75.0,78.5,153.5
30,Deutschland,Jude,Deutsch,Hitler,Opfer,Berlin,Nazi,Pol,Krieg,Nationalsozialismus,0.716404,10,71.0,78.5,149.5
58,Akademie,Potsdam,Sachs,Magdeburg,Akademie der Wissenschaft,Kirche,Sachsen-Anhalt,Thüring,Erfurt,Brandenburg,0.703753,10,69.0,78.5,147.5
83,Prozent,Euro,Bank,Unternehmen,Million,Geld,Anleger,Deutschland,Fond,Aktie,0.690818,10,67.0,78.5,145.5


In [5]:
l.iloc[t.index, :]

,label0,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10,label11,label12,label13,label14,label15,label16,label17,label18,label19
24,Wildschwein,Wildtier,Hauskatze,Tayra,Schaf,Nutztier,klein indisch Zibetkatze,Kulturflüchter,Kleinsäuger,Haustier,Dingo,Kusus,Riesenbeutelmarder,Haushund,Beutegreifer,Lungenwurm,Kaninchen,Fleischfresser,indisch Gazelle,Pampashirsch
12,Duett,Begleitband,Aufführung,Konzert,Tournee,klassisch Musik,Rockband,Auftritt,Bühnenwerk,Show,Singer-Songwriter,Bühnenmusik,Darsteller,Musik sein,Arrangeur,Liedermacher,Gastspiel,Soundtrack,Gitarrist,Musiker
68,Gestirn,Sonnensystem,Mond,Planetensystem,interplanetar Staub,Raumsonde,Umlaufbahn,Infrarotgalaxie,Standardkerze,Umlaufzeit,interstellar Medium,Exoplanet,Ross 128 B,lokal Blas,Sternstrom,Hipparcos-Katalog,Himmelskörper,Epsilon Eridani,Sternhaufen,Trappist-1
90,Jesus Christus,Gott,Wort Gott,Sohn Gott,Leib Christus,Anbetung,Bekehrung,Gottesfurcht,Christentum,heilig Schrift,Evangelium,Glaubensbekenntnis,Reich Gott,Glaube,himmlisch König,Sünde,Abrahams Schoß,Offenbarungsreligion,Heiland,Heilige
45,Sonntagabend,Freitag,Donnerstag,Dienstag,Sonntag,Samstag,Mittwoch,Wochenende,Feiertag,Vormittag,Tulpensonntag,heute mittag,Hallo Niedersachse,Wochentag,Werktag,Servicezeit,RTL Nachtjournal,Sat.1 Nachricht,Zdf-Mittagsmagazin,heute – in Europa
6,päpstlich Legat,Weihbischof,Kurienbischof,Erzbistum Turin,Suffraganbischof,Erzbischof,Erzpriester,Diözesanbischof,Bischof,apostolisch Delegat,Metropolit,Dompropst,Pontifikat,Papst,Bischofssitz,Offizial,Diözese,Generalvikar,Kathedrale,James Robert Knox
32,vereinigen Staat,Staat,Siegermacht,Militärbündnis,westlich Welt,Regierungschef,Land,Nachfolgestaat,Kolonialmacht,Landmacht,Supermacht,Nationalstaat,Iran,allgemein Gewaltverbot,Westeuropa,nah Osten,Mutterland,Monarchie,arabisch Welt,Syrien
30,Zeit der Nationalsozialismus,Holocaust,Antisemitismus,dritte Reich,Überfall auf Pol,Volksdeutscher,Nationalsozialismus,Anschluss Österreich,zweite Weltkrieg,Novemberpogrom 1938,Adolf Hitler,protestantisch Rompilger,volksdeutscher Selbstschutz,polnisch Aufstand,Nazi,Konzentrationslager,Machtergreifung,nürnberger Gesetz,nationalsozialistisch Propaganda,Besatzungsmacht
58,Thüring,Merseburg,Erfurt,Sachsen-Anhalt,Provinz Sachs,Brandenburg,Jena,Leipzig,Magdeburg,Naumburg,Land Brandenburg,Brandenburg an der Havel,Quedlinburg,Halberstadt,Cottbus,Dietzhausen,Gera,Nordhausen,Chemnitz,Bernburg
83,One Equity Partner,Geldmarktfond,Immobilienfond,Cinven,Startkapital,BC Partner,EQT,Zweitmarkt,Investmentgesellschaft,Advent International,Aktionär,Gazprombank,Refco,Singapore Exchang,Grundkapital,Bankensektor,Beteiligungsgesellschaft,Investmentfond,Die Bank,Pensionsfond


In [7]:
l = load('labels', 'dewac1', 100, 'e42', 'c_npmi', 'minimal')
l

('labels', 'dewac1', 100, 'e42', 'c_npmi', 'small')
labels
dewac_nbfiles01
Reading ../data/preprocessed/LDAmodel/noun/Reranker/dewac_nbfiles01_label-candidates_full.csv


,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10,label11,label12,label13,label14,label15,label16,label17,label18,label19
0,Kommunismus,demokratisch_Sozialismus,Sozialdemokratie,Diktatur_der_Proletariat,sozialistisch_Partei,Sozialismus,Marxismus,bürgerlich_Partei,Arbeiterpartei,gothaer_Programm,Staatssozialismus,Sozialfaschismusthese,kommunistisch_Partei_Deutschland,Leninismus,sozialistisch_Demokratie,Einheitsfront,antiautoritär_Sozialismus,Arbeiterbewegung,Marxismus-Leninismus
1,Stiefmutter,Hedwig,Pittheu,Stiefvater,Schwester,Adelheid,Anna,Johann_von_England,Igrain,Eriphyle,Andrea_von_Ungar,Argeia,Der_Vater,Tochter_der_König,Fürstin,Anna_von_Böhme,Enkel,Großvater,Cousine
2,Lebenspartnerschaftsgesetz,gleichgeschlechtlich,Verpartnerung,Lebensformenpolitik,lesbisch,Homosexualität,Scheidung,Eherecht,Lebensgemeinschaft,Regenbogenfamilie,Eingetragene_Partnerschaft,Ehescheidung,Gütertrennung,Abstandsgebot,Zusammenleben,sexuell_Identität,Partnerschaftsgesetz,Adoption,kirchlich_Trauung
3,Personalvertretungsgesetz,Tarifvertrag,Bundespersonalvertretungsgesetz,Beschäftigte,Betriebsrat,Gesamtbetriebsrat,Tarifvertragspartei,Sprecherausschuss,Arbeitnehmervertretung,Drittelbeteiligungsgesetz,Belegschaft,sozial_Angelegenheit,Arbeitsrecht,Betriebsverfassung,Betriebsrentengesetz,Arbeitsverhältnis,Betriebsbuße,Betriebsänderung,Arbeitsvertrag
4,Aktiengesellschaft,Geschäftsanteil,Grundkapital,Kapitalerhöhung,Stammkapital,Gesellschafter,Gesellschafterversammlung,Anteilseigner,Vorstandsvorsitzende,Stammaktie,Hauptversammlung,Kommanditgesellschaft_auf_Aktie,Generalversammlung,Aufsichtsrat,Gesellschaft_mit_beschränkt_Haftung,Gewerkenversammlung,Kommanditist,Vorzugsaktie,Beteiligungsgesellschaft
5,Verfassung,Staatsrat,Legislative,Nationalversammlung,Volksvertretung,Verfassungsänderung,Gesetzeskraft,Gesetzgebung,Senat,Gesetzentwurf,Staatsgerichtshof,Generalstaatsanwalt,gemeinsam_Ausschuss,Gesetzgebungsverfahren,Abgeordnete,Landesverfassung,Verfassung_der_Land_Baden-Württemberg,Parlament,Verfassungsreform
6,Weihbischof,Kurienbischof,Erzbistum_Turin,Suffraganbischof,Diözesanbischof,Bischof,Erzpriester,Papst,Erzbischof,apostolisch_Delegat,Generalvikar,Kardinal,Dompropst,Metropolit,Diözese,heilig_Stuhl,Koadjutor,Titelkirche,Offizial
7,sofortig_Beschwerde,Prozessgericht,Endurteil,Angeklagte,Zulässigkeit,Anhörung,Hauptverfahren,Teilurteil,Gerichtsverfahren,Prozessurteil,Haftbeschwerde,Hauptsacheverfahren,Sprungrevision,Einspruch,Rechtsmittel,sofortig_Vollziehung,Ablehnungsbescheid,Zwischenurteil,Urteil
8,Strafsache,Verdächtige,Strafkammer,Hauptverfahren,Generalstaatsanwaltschaft,Kläger,Staatsanwaltschaft,Gerichtsverfahren,Tatverdacht,Strafverfolgung,Generalstaatsanwalt,Ermittlungsverfahren,Staatsschutzkammer,Untersuchungsrichter,Freispruch,Staatsanwalt,Beschuldigter,vorsätzlich_Tötung,Strafrichter
9,Wehrpflicht,weiß_Jahrgang,Soldat_auf_Zeit,Wehrersatzdienst,Einberufung,freiwillig_Wehrdienst,Ersatzreserve,Berufssoldat,Zivildienstgesetz,Soldatengesetz,Totalverweigerung,Präsenzdienst,Wehrpflicht_in_Deutschland,Reserveoffizier,Reserveoffizieranwärter,Kriegsdienstverweigerung,Wehrsold,Wehrsoldgesetz,Unteroffizier_auf_Zeit
